In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from google.colab import files
from google.colab import drive
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer
from sklearn.impute import IterativeImputer
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/My Drive/T_no_missing.csv'

# Load the CSV data into a DataFrame
data = pd.read_csv(file_path)
data.head(10)

#remove country for later calculation
data = data.drop(columns=["Country"])
data

,T1992,T1993,T1994,T1995,T1996,T1997,T1998,T1999,T2000,T2001,...,T2011,T2012,T2013,T2014,T2015,T2016,T2017,T2018,T2019,T2020
0,-0.294,0.220,0.430,0.359,-0.116,0.471,0.675,1.198,0.993,1.311,...,1.397,0.223,1.281,0.456,1.093,1.555,1.540,1.544,0.910,0.498
1,0.106,0.076,1.330,-0.172,-0.038,0.075,0.795,0.670,1.065,1.532,...,1.055,1.487,1.333,1.198,1.569,1.464,1.121,2.028,1.675,1.498
2,-0.312,0.552,0.732,0.595,0.846,1.059,1.109,1.476,0.820,1.856,...,1.398,1.147,1.192,1.690,1.121,1.757,1.512,1.210,1.115,1.926
3,0.386,0.174,1.508,1.279,0.570,1.788,1.018,1.055,1.050,1.480,...,1.677,1.265,0.831,1.946,1.690,1.990,1.925,1.919,1.964,2.562
4,0.466,0.256,0.212,0.753,0.370,0.107,1.064,0.417,0.169,0.295,...,0.880,0.552,1.044,0.828,1.331,1.609,0.870,1.395,1.752,1.162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,-1.045,-0.368,0.800,-0.166,0.338,-0.121,0.894,0.900,0.249,0.855,...,0.581,1.043,1.114,0.863,1.326,1.615,0.735,2.007,1.204,1.455
180,0.529,0.315,0.903,1.356,1.353,1.536,1.415,0.935,0.874,1.296,...,1.579,1.344,1.423,1.401,1.510,1.732,2.204,0.942,1.477,2.069
181,0.184,0.348,0.507,0.835,0.355,0.656,0.993,0.783,0.728,0.834,...,0.921,1.074,1.016,1.053,1.412,1.660,1.429,1.290,1.444,1.711
182,0.544,-0.117,0.362,0.761,0.393,0.339,0.876,0.064,-0.040,0.263,...,0.956,0.972,0.790,0.917,1.450,1.401,0.105,0.648,0.855,0.891


In [ ]:
miss_data = pd.read_csv('/content/drive/MyDrive/T_missing.csv')
miss_data_x = miss_data.drop(columns=["Country"])
miss_data_x

,T1992,T1993,T1994,T1995,T1996,T1997,T1998,T1999,T2000,T2001,...,T2011,T2012,T2013,T2014,T2015,T2016,T2017,T2018,T2019,T2020
0,-0.294,0.220,0.430,0.359,-0.116,0.471,0.675,1.198,0.993,1.311,...,1.397,0.223,1.281,0.456,1.093,1.555,1.540,1.544,0.910,0.498
1,0.106,0.076,1.330,-0.172,-0.038,0.075,0.795,0.670,1.065,1.532,...,1.055,1.487,1.333,1.198,1.569,1.464,1.121,2.028,1.675,1.498
2,-0.312,0.552,0.732,0.595,0.846,1.059,1.109,1.476,NaN,1.856,...,1.398,1.147,1.192,1.690,1.121,1.757,1.512,1.210,1.115,1.926
3,0.386,0.174,1.508,1.279,0.570,1.788,1.018,1.055,1.050,1.480,...,1.677,1.265,0.831,1.946,1.690,1.990,1.925,1.919,1.964,2.562
4,0.466,0.256,0.212,0.753,0.370,0.107,1.064,0.417,0.169,0.295,...,0.880,0.552,1.044,0.828,1.331,1.609,0.870,1.395,1.752,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,-1.045,-0.368,0.800,-0.166,0.338,-0.121,0.894,0.900,0.249,0.855,...,0.581,1.043,1.114,0.863,1.326,1.615,0.735,2.007,1.204,1.455
180,0.529,0.315,0.903,1.356,1.353,1.536,1.415,0.935,0.874,1.296,...,1.579,1.344,1.423,1.401,1.510,1.732,2.204,0.942,1.477,2.069
181,0.184,0.348,0.507,0.835,NaN,0.656,0.993,0.783,0.728,NaN,...,0.921,1.074,1.016,1.053,1.412,1.660,1.429,1.290,1.444,1.711
182,0.544,-0.117,0.362,0.761,0.393,0.339,0.876,0.064,-0.040,0.263,...,0.956,0.972,0.790,0.917,1.450,1.401,0.105,0.648,0.855,0.891


# Mean imputation - columnwise

In [ ]:
#impute the mean value in column direction
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(miss_data_x)
SimpleImputer()
imputed_mean_data_x=imp.transform(miss_data_x)
imputed_mean_data_x #array type, after imputation

#put the titles back, convert it into dataframe
imputed_mean_data_x_df = pd.DataFrame(imputed_mean_data_x, columns=['T1992', 'T1993', 'T1994', 'T1995', 'T1996', 'T1997', 'T1998', 'T1999',
       'T2000', 'T2001', 'T2002', 'T2003', 'T2004', 'T2005', 'T2006', 'T2007',
       'T2008', 'T2009', 'T2010', 'T2011', 'T2012', 'T2013', 'T2014', 'T2015',
       'T2016', 'T2017', 'T2018', 'T2019', 'T2020'])
#print out the dataframe after imputing the
imputed_mean_data_x_df

,T1992,T1993,T1994,T1995,T1996,T1997,T1998,T1999,T2000,T2001,...,T2011,T2012,T2013,T2014,T2015,T2016,T2017,T2018,T2019,T2020
0,-0.294,0.220,0.430,0.359,-0.116000,0.471,0.675,1.198,0.993000,1.311000,...,1.397,0.223,1.281,0.456,1.093,1.555,1.540,1.544,0.910,0.498000
1,0.106,0.076,1.330,-0.172,-0.038000,0.075,0.795,0.670,1.065000,1.532000,...,1.055,1.487,1.333,1.198,1.569,1.464,1.121,2.028,1.675,1.498000
2,-0.312,0.552,0.732,0.595,0.846000,1.059,1.109,1.476,0.680063,1.856000,...,1.398,1.147,1.192,1.690,1.121,1.757,1.512,1.210,1.115,1.926000
3,0.386,0.174,1.508,1.279,0.570000,1.788,1.018,1.055,1.050000,1.480000,...,1.677,1.265,0.831,1.946,1.690,1.990,1.925,1.919,1.964,2.562000
4,0.466,0.256,0.212,0.753,0.370000,0.107,1.064,0.417,0.169000,0.295000,...,0.880,0.552,1.044,0.828,1.331,1.609,0.870,1.395,1.752,1.578365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,-1.045,-0.368,0.800,-0.166,0.338000,-0.121,0.894,0.900,0.249000,0.855000,...,0.581,1.043,1.114,0.863,1.326,1.615,0.735,2.007,1.204,1.455000
180,0.529,0.315,0.903,1.356,1.353000,1.536,1.415,0.935,0.874000,1.296000,...,1.579,1.344,1.423,1.401,1.510,1.732,2.204,0.942,1.477,2.069000
181,0.184,0.348,0.507,0.835,0.263209,0.656,0.993,0.783,0.728000,0.873792,...,0.921,1.074,1.016,1.053,1.412,1.660,1.429,1.290,1.444,1.711000
182,0.544,-0.117,0.362,0.761,0.393000,0.339,0.876,0.064,-0.040000,0.263000,...,0.956,0.972,0.790,0.917,1.450,1.401,0.105,0.648,0.855,0.891000


# Calculate the MSE, RMSE, MAE of the Mean Imputation Method

In [ ]:
# Mask of where values were missing
data=data.to_numpy() #convert data(dataframe) to data(numpy array) for calculation purpose
mask = np.isnan(miss_data_x)
true_values = data[mask]
imputed_values = imputed_mean_data_x[mask]

#calculate the MSE
mse = mean_squared_error(true_values, imputed_values)
print(f"Mean Squred Error (MSE) of mean imputation: {mse}")

#calculate the RMSE
rmse = np.sqrt(mean_squared_error(true_values, imputed_values))
print(f"Root Mean Squred Error (RMSE) of mean imputation: {rmse}")

# Calculate MAE only for the imputed values
mae = mean_absolute_error(true_values, imputed_values)
print(f"Mean Absolute Error (MAE) of mean imputation: {mae}")


Mean Squred Error (MSE) of mean imputation: 0.2603474065157438
Root Mean Squred Error (RMSE) of mean imputation: 0.5102424977554729
Mean Absolute Error (MAE) of mean imputation: 0.3895309740586603


In [ ]:
type(true_values)

numpy.ndarray

# Median imputation - columnwise

In [ ]:
#impute the mean value in column direction
imp = SimpleImputer(missing_values=np.nan, strategy='median')
imp.fit(miss_data_x)
SimpleImputer()
imputed_median_data_x=imp.transform(miss_data_x)
imputed_median_data_x #array type, after imputation

#put the titles back, convert it into dataframe
imputed_median_data_x_df = pd.DataFrame(imputed_median_data_x, columns=['T1992', 'T1993', 'T1994', 'T1995', 'T1996', 'T1997', 'T1998', 'T1999',
       'T2000', 'T2001', 'T2002', 'T2003', 'T2004', 'T2005', 'T2006', 'T2007',
       'T2008', 'T2009', 'T2010', 'T2011', 'T2012', 'T2013', 'T2014', 'T2015',
       'T2016', 'T2017', 'T2018', 'T2019', 'T2020'])
#print out the dataframe after imputing the
imputed_median_data_x_df

,T1992,T1993,T1994,T1995,T1996,T1997,T1998,T1999,T2000,T2001,...,T2011,T2012,T2013,T2014,T2015,T2016,T2017,T2018,T2019,T2020
0,-0.294,0.220,0.430,0.359,-0.116,0.471,0.675,1.198,0.993,1.311,...,1.397,0.223,1.281,0.456,1.093,1.555,1.540,1.544,0.910,0.4980
1,0.106,0.076,1.330,-0.172,-0.038,0.075,0.795,0.670,1.065,1.532,...,1.055,1.487,1.333,1.198,1.569,1.464,1.121,2.028,1.675,1.4980
2,-0.312,0.552,0.732,0.595,0.846,1.059,1.109,1.476,0.554,1.856,...,1.398,1.147,1.192,1.690,1.121,1.757,1.512,1.210,1.115,1.9260
3,0.386,0.174,1.508,1.279,0.570,1.788,1.018,1.055,1.050,1.480,...,1.677,1.265,0.831,1.946,1.690,1.990,1.925,1.919,1.964,2.5620
4,0.466,0.256,0.212,0.753,0.370,0.107,1.064,0.417,0.169,0.295,...,0.880,0.552,1.044,0.828,1.331,1.609,0.870,1.395,1.752,1.4825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,-1.045,-0.368,0.800,-0.166,0.338,-0.121,0.894,0.900,0.249,0.855,...,0.581,1.043,1.114,0.863,1.326,1.615,0.735,2.007,1.204,1.4550
180,0.529,0.315,0.903,1.356,1.353,1.536,1.415,0.935,0.874,1.296,...,1.579,1.344,1.423,1.401,1.510,1.732,2.204,0.942,1.477,2.0690
181,0.184,0.348,0.507,0.835,0.286,0.656,0.993,0.783,0.728,0.728,...,0.921,1.074,1.016,1.053,1.412,1.660,1.429,1.290,1.444,1.7110
182,0.544,-0.117,0.362,0.761,0.393,0.339,0.876,0.064,-0.040,0.263,...,0.956,0.972,0.790,0.917,1.450,1.401,0.105,0.648,0.855,0.8910


# Calculate the MSE, RMSE, MAE of the Median Imputation Method

In [ ]:
# Mask of where values were missing
mask = np.isnan(miss_data_x)
true_values = data[mask]
imputed_values = imputed_median_data_x[mask]

#calculate the MSE
mse = mean_squared_error(true_values, imputed_values)
print(f"Mean Squred Error (MSE) of median imputation: {mse}")

#calculate the RMSE
rmse = np.sqrt(mean_squared_error(true_values, imputed_values))
print(f"Root Mean Squred Error (RMSE) of median imputation: {rmse}")

# Calculate MAE only for the imputed values
mae = mean_absolute_error(true_values, imputed_values)
print(f"Mean Absolute Error (MAE) of median imputation: {mae}")

Mean Squred Error (MSE) of median imputation: 0.26732388403041824
Root Mean Squred Error (RMSE) of median imputation: 0.5170337358726393
Mean Absolute Error (MAE) of median imputation: 0.3862813688212928


# Multivariate feature imputation-Column wise


In [ ]:
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit(miss_data_x)
IterativeImputer(random_state=0)
multivariate_impute_data_x=imp.transform(miss_data_x)
multivariate_impute_data_x

#put the titles back, convert it into dataframe
multivariate_impute_data_x_df = pd.DataFrame(multivariate_impute_data_x, columns=['T1992', 'T1993', 'T1994', 'T1995', 'T1996', 'T1997', 'T1998', 'T1999',
       'T2000', 'T2001', 'T2002', 'T2003', 'T2004', 'T2005', 'T2006', 'T2007',
       'T2008', 'T2009', 'T2010', 'T2011', 'T2012', 'T2013', 'T2014', 'T2015',
       'T2016', 'T2017', 'T2018', 'T2019', 'T2020'])

#print out the dataframe after imputing the
multivariate_impute_data_x_df

,T1992,T1993,T1994,T1995,T1996,T1997,T1998,T1999,T2000,T2001,...,T2011,T2012,T2013,T2014,T2015,T2016,T2017,T2018,T2019,T2020
0,-0.294,0.220,0.430,0.359,-0.116000,0.471,0.675,1.198,0.993000,1.311000,...,1.397,0.223,1.281,0.456,1.093,1.555,1.540,1.544,0.910,0.498000
1,0.106,0.076,1.330,-0.172,-0.038000,0.075,0.795,0.670,1.065000,1.532000,...,1.055,1.487,1.333,1.198,1.569,1.464,1.121,2.028,1.675,1.498000
2,-0.312,0.552,0.732,0.595,0.846000,1.059,1.109,1.476,0.994762,1.856000,...,1.398,1.147,1.192,1.690,1.121,1.757,1.512,1.210,1.115,1.926000
3,0.386,0.174,1.508,1.279,0.570000,1.788,1.018,1.055,1.050000,1.480000,...,1.677,1.265,0.831,1.946,1.690,1.990,1.925,1.919,1.964,2.562000
4,0.466,0.256,0.212,0.753,0.370000,0.107,1.064,0.417,0.169000,0.295000,...,0.880,0.552,1.044,0.828,1.331,1.609,0.870,1.395,1.752,1.308182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,-1.045,-0.368,0.800,-0.166,0.338000,-0.121,0.894,0.900,0.249000,0.855000,...,0.581,1.043,1.114,0.863,1.326,1.615,0.735,2.007,1.204,1.455000
180,0.529,0.315,0.903,1.356,1.353000,1.536,1.415,0.935,0.874000,1.296000,...,1.579,1.344,1.423,1.401,1.510,1.732,2.204,0.942,1.477,2.069000
181,0.184,0.348,0.507,0.835,0.361339,0.656,0.993,0.783,0.728000,0.883945,...,0.921,1.074,1.016,1.053,1.412,1.660,1.429,1.290,1.444,1.711000
182,0.544,-0.117,0.362,0.761,0.393000,0.339,0.876,0.064,-0.040000,0.263000,...,0.956,0.972,0.790,0.917,1.450,1.401,0.105,0.648,0.855,0.891000


# Calculate the MSE, RMSE, MAE of the multivariate feature Imputation Method

In [ ]:
# Mask of where values were missing
mask = np.isnan(miss_data_x)
true_values = data[mask]
imputed_values = multivariate_impute_data_x[mask]

#calculate the MSE
mse = mean_squared_error(true_values, imputed_values)
print(f"Mean Squred Error (MSE) of multivariate feature imputation: {mse}")

#calculate the RMSE
rmse = np.sqrt(mean_squared_error(true_values, imputed_values))
print(f"Root Mean Squred Error (RMSE) of multivariate feature imputation: {rmse}")

# Calculate MAE only for the imputed values
mae = mean_absolute_error(true_values, imputed_values)
print(f"Mean Absolute Error (MAE) of multivariate feature imputation: {mae}")

Mean Squred Error (MSE) of multivariate feature imputation: 0.05396066266665021
Root Mean Squred Error (RMSE) of multivariate feature imputation: 0.23229434488736528
Mean Absolute Error (MAE) of multivariate feature imputation: 0.17313393062861676


# Nearest neighbors imputation-Column wise

In [ ]:
imputer = KNNImputer(n_neighbors=2, weights="uniform")
KNN_impute_data_x=imputer.fit_transform(miss_data_x)

#put the titles back, convert it into dataframe
KNN_impute_data_x_df = pd.DataFrame(KNN_impute_data_x, columns=['T1992', 'T1993', 'T1994', 'T1995', 'T1996', 'T1997', 'T1998', 'T1999',
       'T2000', 'T2001', 'T2002', 'T2003', 'T2004', 'T2005', 'T2006', 'T2007',
       'T2008', 'T2009', 'T2010', 'T2011', 'T2012', 'T2013', 'T2014', 'T2015',
       'T2016', 'T2017', 'T2018', 'T2019', 'T2020'])

#print out the dataframe after imputing the
KNN_impute_data_x_df

,T1992,T1993,T1994,T1995,T1996,T1997,T1998,T1999,T2000,T2001,...,T2011,T2012,T2013,T2014,T2015,T2016,T2017,T2018,T2019,T2020
0,-0.294,0.220,0.430,0.359,-0.116,0.471,0.675,1.198,0.9930,1.3110,...,1.397,0.223,1.281,0.456,1.093,1.555,1.540,1.544,0.910,0.4980
1,0.106,0.076,1.330,-0.172,-0.038,0.075,0.795,0.670,1.0650,1.5320,...,1.055,1.487,1.333,1.198,1.569,1.464,1.121,2.028,1.675,1.4980
2,-0.312,0.552,0.732,0.595,0.846,1.059,1.109,1.476,0.9295,1.8560,...,1.398,1.147,1.192,1.690,1.121,1.757,1.512,1.210,1.115,1.9260
3,0.386,0.174,1.508,1.279,0.570,1.788,1.018,1.055,1.0500,1.4800,...,1.677,1.265,0.831,1.946,1.690,1.990,1.925,1.919,1.964,2.5620
4,0.466,0.256,0.212,0.753,0.370,0.107,1.064,0.417,0.1690,0.2950,...,0.880,0.552,1.044,0.828,1.331,1.609,0.870,1.395,1.752,1.5895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,-1.045,-0.368,0.800,-0.166,0.338,-0.121,0.894,0.900,0.2490,0.8550,...,0.581,1.043,1.114,0.863,1.326,1.615,0.735,2.007,1.204,1.4550
180,0.529,0.315,0.903,1.356,1.353,1.536,1.415,0.935,0.8740,1.2960,...,1.579,1.344,1.423,1.401,1.510,1.732,2.204,0.942,1.477,2.0690
181,0.184,0.348,0.507,0.835,0.557,0.656,0.993,0.783,0.7280,0.7855,...,0.921,1.074,1.016,1.053,1.412,1.660,1.429,1.290,1.444,1.7110
182,0.544,-0.117,0.362,0.761,0.393,0.339,0.876,0.064,-0.0400,0.2630,...,0.956,0.972,0.790,0.917,1.450,1.401,0.105,0.648,0.855,0.8910


# Calculate the MSE, RMSE, MAE of the KNN Imputation Method

In [ ]:
# Mask of where values were missing
mask = np.isnan(miss_data_x)
true_values = data[mask]
imputed_values = KNN_impute_data_x[mask]

#calculate the MSE
mse = mean_squared_error(true_values, imputed_values)
print(f"Mean Squred Error (MSE) of Nearest neighbors imputation imputation: {mse}")

#calculate the RMSE
rmse = np.sqrt(mean_squared_error(true_values, imputed_values))
print(f"Root Mean Squred Error (RMSE) of Nearest neighbors imputation imputation: {rmse}")

# Calculate MAE only for the imputed values
mae = mean_absolute_error(true_values, imputed_values)
print(f"Mean Absolute Error (MAE) of Nearest neighbors imputation imputation: {mae}")

Mean Squred Error (MSE) of Nearest neighbors imputation imputation: 0.05781258174904943
Root Mean Squred Error (RMSE) of Nearest neighbors imputation imputation: 0.24044247076806008
Mean Absolute Error (MAE) of Nearest neighbors imputation imputation: 0.16486692015209128
